In [ ]:
from pprint import pprint
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import os
import sys
sys.path.append('..')
from util.signature_util import SignatureVisualizer

In [ ]:
case = 0 # select one of [0, 1, 2, 3]
data_root = '../../datasets'

In [ ]:
signature_template_path = '../info'
if case == 0:
    dataset_name = 'FlickrCI3D_signature'
    subset = 'train'
    im_name = 'boys_23866'
    signature_id = 0
elif case == 1:
    dataset_name = 'FlickrSC3D_signature'
    subset = 'train'
    im_name = 'friends_15971' 
    signature_id = 0
elif case == 2:
    dataset_name = 'FlickrCI3D_classification'
    subset = 'train'
    im_name = 'boys_23866' 
    signature_id = 0
elif case == 3:
    dataset_name = 'FlickrSC3D_classification'
    subset = 'train'
    im_name = 'boys_52282' 
    signature_id = 0

In [ ]:
dataset_to_ann_name = {
    'FlickrCI3D_signature': 'interaction_contact_signature',
    'FlickrCI3D_classification': 'interaction_contact_classification',
    'FlickrSC3D_signature': 'self_contact_signature',
    'FlickrSC3D_classification': 'self_contact_classification'
}
dataset_to_ann_type = {
    'FlickrCI3D_signature': 'ci_sign',
    'FlickrCI3D_classification': 'ci_classif',
    'FlickrSC3D_signature': 'sc_sign',
    'FlickrSC3D_classification': 'sc_classif'
}
contact_type_to_message = {0: "no contact", 1: "uncertain contact", 2: "contact"}
ann_name = dataset_to_ann_name[dataset_name]
ann_type = dataset_to_ann_type[dataset_name]

In [ ]:
subset_fn = os.path.join(data_root, dataset_name, subset)
ann_fn = '%s/%s.json' % (subset_fn, ann_name)
images_dir = '%s/images' % subset_fn

with open(ann_fn, 'rb') as f:
    anns = json.load(f)

In [ ]:
im_fn = '%s/%s.png' % (images_dir, im_name)
img = mpimg.imread(im_fn)

ann = anns[im_name]

if dataset_name.endswith('signature'):
    ghum_signature = ann[ann_type][signature_id]['ghum']
    smplx_signature = ann[ann_type][signature_id]['smplx']
else:
    contact_type = ann[ann_type][signature_id]['contact_type']
    print(im_name, contact_type_to_message[contact_type])
person_ids = [ann[ann_type][signature_id]['person_id']] if 'person_id' in ann[ann_type][signature_id] else ann[ann_type][signature_id]['person_ids']
bbxes = [ann['bbxes'][person_id] for person_id in person_ids]

plt.figure(figsize=(20, 20))
plt.imshow(img)
ax = plt.gca()

colors = ['red', 'blue']
for i, bbx in enumerate(bbxes):
    rect = patches.Rectangle((bbx[0], bbx[1]), bbx[2]-bbx[0], bbx[3]-bbx[1], 
                             linewidth=4, edgecolor=colors[i], facecolor='none')
    ax.add_patch(rect)
plt.show()

# GHUM Signature Visualizer

In [ ]:
GHUM_Models_Path = '../../ghumrepo/ghum/shmodels'
ghum_signature_visualizer = SignatureVisualizer(path_to_template=signature_template_path, 
                                      model_type='GHUM', 
                                      models_path=GHUM_Models_Path)

if dataset_name == 'FlickrSC3D_signature':
    scene = ghum_signature_visualizer.self_contact_signature(ghum_signature)
elif dataset_name == 'FlickrCI3D_signature':
    scene = ghum_signature_visualizer.interaction_contact_signature(ghum_signature)
scene.show()

# SMPLX Signature Visualizer

In [ ]:
SMPLX_Models_Path = '../../smplx_body_models'
smplx_signature_visualizer = SignatureVisualizer(path_to_template=signature_template_path, 
                                      model_type='SMPLX', 
                                      models_path=SMPLX_Models_Path)

if dataset_name == 'FlickrSC3D_signature':
    scene = smplx_signature_visualizer.self_contact_signature(ghum_signature)
elif dataset_name == 'FlickrCI3D_signature':
    scene = smplx_signature_visualizer.interaction_contact_signature(ghum_signature)
scene.show()